# 自動微分
Pytorchで、計算履歴を保存し、自動で微分操作を実現するAutoGrad(自動微分)の説明。

# Automatic Differentiation with `torch.autograd`
ニューラルネットワークを訓練する際に、その学習アルゴリズムとして、  
基本的にbackpropagationが使用される。  
このとき、モデルの重みやパラメータの損失関数に対する微分値に応じて調整される。  
これらの勾配を自動的に計算する仕組みが自動微分であり、`torch.autograd`に組み込まれている。

以下、具体例として  
入力を`x`、パラメータを`w`, `b`とする。

In [ ]:
import torch

In [ ]:
x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5,3, requires_grad=True)
b = torch.randn(3, requires_grad=True)

z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

# テンソル、関数、計算グラフの関係
上記のコードは次の計算グラフ(**computational graph)**を示している。  

<img src="https://pytorch.org/tutorials/_images/comp-graph.png" width=50% alt="計算グラフ" title="計算グラフ">

上記のネットーワークのうち、`w`, `b`が最適化したいパラメータとなる。  
そのために、**これらの変数に対する損失関数の微分**を計算する必要がある。  
これらのパラメータでの微分を計算するために、`requires_grad=True`を設定する必要がある。  

#### 補足
`requires_grad`は、
1. テンソルを定義する際、引数に`requires_grad=True`と指定する。  
2. 変数定義後、`x.requires_grad_(True)`を実行する。  

の二つの方法のどちらかで設定する必要がある。  

#### 補足
計算グラフを構築する際に、テンソルに適用する関数は、実際には`Function`クラスのオブジェクト。  
これらのオブジェクトでは、以下の二つが定義されている。  
1. 順伝播時に、入力をどのように処理するのか。  
2. 逆伝播時に、勾配をどのように計算するのか。  

さらに、勾配は、テンソルの`grad_fn`プロパティに格納されている。  
`Function`の詳細は、[Functionの詳細](https://pytorch.org/docs/stable/autograd.html#function)

In [ ]:
print("Gradient function for z = ", z.grad_fn)
print("Gradient function for loss = ", loss.grad_fn)

# 勾配の計算
ニューラルネットワークの各パラメータを最適化することを考える。  
入力x, 出力yが与えられたもとで、損失関数の各変数の偏微分  

$\frac{\partial loss}{\partial w}$
$\frac{\partial loss}{\partial b}$  

を求める必要がある。  
**これらの偏微分を求める**ために、`loss.backward()`を実行し、  
各変数の偏微分の値、`w.grad`と`b.grad`を求める。

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

#### 補足
`grad`が求められるのは以下の二つを満たす時のみ。  
1. 計算グラフのleaf nodeであること  
2. その変数の`requires_grad=True`であること  

全ての変数で勾配が計算可能でないことに注意。  

#### 補足
勾配の計算は、各計算グラフに対して、**`backward`を実行後、一度のみ**計算できる。  
同じ計算グラフに対して、複数回の`backward`を実行したい場合には、  
`backward`実行時に、`retain_graph=True`を引数として渡す必要がある。

# 勾配計算が不必要なとき
デフォルトでは、`requires_grad=True`である全てのテンソルは計算履歴が保持され、微分計算が可能。  
しかし、訓練済みのモデルでの推論時や順伝播のみを行うときなど、計算が不要。  

実装コードで勾配計算を不要にするには以下の二つの方法がある  
1. `torch.no_grad()`のブロック内に計算を記述する。  
2. テンソルに`detach()`を使用する  

実際に、勾配の計算を不要にするケースは以下のようなときがある。  
1. ネットワークの一部のパラメータを固定したい
> ファインチューニング時によくあるケース  
2. 順伝播の計算スピードを高速化したいケース  

In [ ]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

In [ ]:
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

#### 計算グラフの補足
複雑な内容だったので、元記事を一旦丸写ししておく...

計算グラフについて補足
----------------------------

概念的には、autogradはテンソルとそれらに対する演算処理を[`Function`](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)を構成単位として、DAG（a directed acyclic graph）の形で保存したグラフです。


DAGにおいて、各leafは入力テンソル、そしてrootは出力テンソルです。

このグラフをrootから各leafまでchain rule（微分の連鎖律）で追跡することによって各変数に対する偏微分の値を求めることができます。



順伝搬では autograd は2つの処理を同時に行っています。

- 指定された演算を実行し、計算結果のテンソルを求める
- DAGの各操作の*gradient function* を更新する

<br>

逆伝搬では、``.backward()``がDAGのrootのテンソルに対して実行されると、autogradは、

- 各変数の ``.grad_fn``を計算する
- 各変数の``.grad``属性に微分値を代入する
- 微分の連鎖律を使用して、各leafのテンソルの微分値を求める

を行います。

【注意】

PyTorchではDAGは動的です（Functionで計算処理される際に逐次構築されていきます）。

そして、`.backward()`を呼び出すたびに、autogradは再度新しいグラフを作成します。

この特性こそが、モデルの順伝搬時に制御フロー文（if文やfor文）を使える理由であり、必要に応じて反復ごとに形や大きさ、操作を変えることができます。



【日本語訳注】

上記内容は初心者の方には非常に難しい話です。

PyTorchは Define-by-run 形式であり、事前に計算グラフを定義するのではなく、計算を実行する際に、柔軟に計算グラフを作ってくれます。

一方で、Define-and-run形式のディープラーニングフレームワーク（例えば、TensorFlow v1など）は、事前に計算グラフを定義する必要があるため、for文やif文といった制御フローの構文を柔軟に使いづらいです。

このことを上記内容では説明しています。

補注：テンソルに対する勾配とヤコビ行列
--------------------------------------

多くの場合、スカラー値を出力する損失関数に対して、とある変数の勾配を計算します。

ですが、関数の出力がスカラー値ではなく、任意のテンソルであるケースもあります。

このような場合、PyTorchでは実際の勾配ではなく、いわゆるヤコビ行列（Jacobian
matrix）を計算することができます。


ベクトル関数 $\vec{y}=f(\vec{x})$,において、
$\vec{x}=\langle x_1,\dots,x_n\rangle$ 、そして　$\vec{y}=\langle y_1,\dots,y_m\rangle$の場合、

その勾配、 $\vec{y}$ with respect to $\vec{x}$ はヤコビ行列 で与えられます。

\begin{split}\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}\end{split}

ヤコビ行列そのものを計算する代わりにPyTorchでは、**Jacobian Product**、 $v^T\cdot J$ を、入力ベクトル$v=(v_1 \dots v_m)$ に対して計算します。

これは、$v$を引数として ``backward``メソッドを呼び出すことで計算されます。

なお$v$ のサイズは積を計算したい、元のテンソルの大きさと同じである必要があります。


In [ ]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

上記において、同じ変数`inp`に対して、``backward``を2回目実行した際には、勾配が異なる値になった点に注意してください。

これはPyTorchでは``backward``を実行すると、勾配を蓄積（accumulate）する仕様だからです。

すなわち、計算グラフの全leafの``grad``には、勾配が足し算されます。

<br>

そのため適切に勾配を計算するには、``grad``を事前に0にリセットする必要があります。

なお実際にPyTorchでディープラーニングモデルの訓練を行う際には、**オプティマイザー（optimizer）**が、勾配をリセットする役割を担ってくれます

【注意】

本チュートリアルの最初の方で、``backward()``関数を引数のパラメータなしに実行していました。

これは実質的には、``backward(torch.tensor(1.0))``を実行しているのと同じとなります。

``backward()``はスカラー値の関数（例えば訓練時の損失関数）に対して、各パラメータの勾配を計算する際に便利です。

## 詳細
より詳細には、[Aitograd Mechanics](https://pytorch.org/docs/stable/notes/autograd.html)を参照